# �� 니켈 가격 예측 - sparta2 이후 성능 개선 시도

## 🎯 목표
**sparta2에서 달성한 Hybrid (Naive*0.8 + GB*0.2) RMSE 406.80을 개선**

## 📋 실험 목록

### Part A: 피처 엔지니어링 (5가지)
1. Realized Volatility (다중 윈도우)
2. Momentum Indicators (RSI, ROC)
3. Mean Reversion (Z-score, Bollinger)
4. Cross-Asset Features (금속간 상관)
5. Lag Features (1~4주 시차)

### Part B: 모델 다양화 (5가지)  
1. 다양한 Boosting (XGB, LGB, CatBoost)
2. Ridge/ElasticNet Regression
3. ARIMA (통계 모델)
4. Damped Naive (감쇠 추세)
5. 다양한 Hybrid 가중치

### Part C: 고급 기법 (3가지)
1. Regime Detection (적응형 가중치)
2. Stacking Ensemble
3. Hyperparameter Tuning (Grid Search)

### Part D: 검증 및 분석
1. Time Series CV (5-Fold)
2. 최종 비교표
3. 실패 원인 분석

## ⚠️ Data Leakage 방지
- 모든 피처에 `shift(1)` 적용
- Train(~08/03) → Val(08/04~10/20) → Test(10/27~01/12)



In [ ]:
# 환경 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

try:
    from catboost import CatBoostRegressor
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False
    print("CatBoost not available")

try:
    from statsmodels.tsa.arima.model import ARIMA
    ARIMA_AVAILABLE = True
except:
    ARIMA_AVAILABLE = False
    print("ARIMA not available")

# 상수
SPARTA2_RMSE = 406.80  # sparta2 기준선
np.random.seed(42)

CONFIG = {
    'data_file': 'data_weekly_260120.csv',
    'target_col': 'Com_LME_Ni_Cash',
    'val_start': '2025-08-04',
    'val_end': '2025-10-20',
    'test_start': '2025-10-27',
    'test_end': '2026-01-12',
}

# 결과 저장
all_results = []

def add_result(name, rmse, category):
    improvement = SPARTA2_RMSE - rmse
    status = '✅ 개선' if improvement > 0 else '❌ 악화'
    print(f'  {name}: RMSE {rmse:.2f} ({status} {improvement:+.2f})')
    all_results.append({'Model': name, 'RMSE': rmse, 'Category': category, 'Improvement': improvement})

print('='*60)
print('환경 설정 완료')
print(f'기준선: sparta2 Hybrid RMSE = {SPARTA2_RMSE}')
print('='*60)



: 

---
## 🔄 기준선 재현

sparta2의 결과를 먼저 재현하여 공정한 비교 기준 확보


In [ ]:
# 데이터 로드 + 기준선 재현
print('='*60)
print('기준선 재현')
print('='*60)

df = pd.read_csv(CONFIG['data_file'])
df['dt'] = pd.to_datetime(df['dt'])
df = df.set_index('dt').sort_index()

target_col = CONFIG['target_col']
price = df[target_col].copy()

# 분할
train = df[df.index < CONFIG['val_start']]
val = df[(df.index >= CONFIG['val_start']) & (df.index <= CONFIG['val_end'])]
test = df[(df.index >= CONFIG['test_start']) & (df.index <= CONFIG['test_end'])]

y_train, y_val, y_test = train[target_col], val[target_col], test[target_col]
print(f'Train: {len(train)}주 | Val: {len(val)}주 | Test: {len(test)}주')

# Naive Drift 계산
def calc_naive_drift(indices):
    preds = []
    for idx in indices:
        loc = price.index.get_loc(idx)
        if loc >= 2:
            drift = price.iloc[loc-1] - price.iloc[loc-2]
            preds.append(price.iloc[loc-1] + drift)
        else:
            preds.append(price.iloc[loc-1])
    return np.array(preds)

naive_val = calc_naive_drift(val.index)
naive_test = calc_naive_drift(test.index)

# 피처 준비
features = [c for c in train.columns if c != target_col and train[c].dtype in ['float64', 'int64']]
features = [f for f in features if not train[f].isna().all()]

X_train = train[features].fillna(method='ffill').fillna(method='bfill')
X_val = val[features].fillna(method='ffill').fillna(method='bfill')
X_test = test[features].fillna(method='ffill').fillna(method='bfill')

# 기준선 GradientBoosting
gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb.fit(X_train, y_train)
gb_val = gb.predict(X_val)
gb_test = gb.predict(X_test)

# 기준선 Hybrid
baseline_hybrid = 0.8 * naive_test + 0.2 * gb_test
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_hybrid))

print(f'\n🎯 기준선 재현: RMSE {baseline_rmse:.2f}')
print(f'   sparta2 결과: {SPARTA2_RMSE}')
print(f'   차이: {baseline_rmse - SPARTA2_RMSE:+.2f}')

add_result('Baseline (sparta2)', baseline_rmse, 'Baseline')



---
# Part A: 피처 엔지니어링

새로운 피처를 추가하여 ML 모델의 예측력 향상 시도


In [ ]:
# A1. Realized Volatility (다중 윈도우)
print('='*60)
print('A1. Realized Volatility')
print('='*60)

df_fe = df.copy()
ret = np.log(price / price.shift(1))

for w in [4, 8, 12, 26, 52]:
    df_fe[f'RV_{w}w'] = ret.rolling(w).std() * np.sqrt(52)
    df_fe[f'RV_{w}w'] = df_fe[f'RV_{w}w'].shift(1)  # Data Leakage 방지

print(f'추가된 피처: RV_4w, RV_8w, RV_12w, RV_26w, RV_52w')



In [ ]:
# A2. Momentum Indicators
print('='*60)
print('A2. Momentum Indicators')
print('='*60)

# ROC (Rate of Change)
for w in [4, 12, 26]:
    df_fe[f'ROC_{w}w'] = price.pct_change(w).shift(1)

# RSI 계산
def calc_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / (loss + 1e-10)
    return 100 - (100 / (1 + rs))

df_fe['RSI_14'] = calc_rsi(price, 14).shift(1)

print('추가된 피처: ROC_4w, ROC_12w, ROC_26w, RSI_14')



In [ ]:
# A3. Mean Reversion (Z-score, Bollinger)
print('='*60)
print('A3. Mean Reversion')
print('='*60)

for w in [12, 26, 52]:
    ma = price.rolling(w).mean()
    std = price.rolling(w).std()
    df_fe[f'zscore_{w}w'] = ((price - ma) / (std + 1e-8)).shift(1)
    df_fe[f'boll_pos_{w}w'] = ((price - (ma - 2*std)) / (4*std + 1e-8)).shift(1)

print('추가된 피처: zscore, boll_pos (12w, 26w, 52w)')



In [ ]:
# A4. Cross-Asset Features (금속간 상관)
print('='*60)
print('A4. Cross-Asset Features')
print('='*60)

# 다른 금속이 있다면 상관관계 피처 추가
metal_cols = [c for c in df.columns if 'Com_LME' in c and c != target_col]
print(f'관련 금속 컬럼: {metal_cols[:5]}...')

for col in metal_cols[:3]:  # 상위 3개만
    if not df[col].isna().all():
        df_fe[f'corr_Ni_{col[-5:]}'] = price.rolling(12).corr(df[col]).shift(1)

print('금속간 12주 rolling 상관계수 추가')



In [ ]:
# A5. Lag Features (1~4주 시차)
print('='*60)
print('A5. Lag Features')
print('='*60)

for lag in [1, 2, 3, 4]:
    df_fe[f'price_lag_{lag}'] = price.shift(lag)
    df_fe[f'ret_lag_{lag}'] = ret.shift(lag)

print('추가된 피처: price_lag_1~4, ret_lag_1~4')



In [ ]:
# 피처 엔지니어링 적용 후 테스트
print('='*60)
print('피처 엔지니어링 종합 테스트')
print('='*60)

# 새 피처로 데이터셋 재구성
train_fe = df_fe[df_fe.index < CONFIG['val_start']].dropna()
test_fe = df_fe[(df_fe.index >= CONFIG['test_start']) & (df_fe.index <= CONFIG['test_end'])].dropna()

features_fe = [c for c in train_fe.columns if c != target_col and train_fe[c].dtype in ['float64', 'int64']]
X_train_fe = train_fe[features_fe].fillna(0)
X_test_fe = test_fe[features_fe].fillna(0)

# 새 피처로 GB 재학습
gb_fe = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb_fe.fit(X_train_fe, train_fe[target_col])
pred_fe = gb_fe.predict(X_test_fe)

# Hybrid
naive_test_fe = calc_naive_drift(test_fe.index)
hybrid_fe = 0.8 * naive_test_fe + 0.2 * pred_fe
rmse_fe = np.sqrt(mean_squared_error(test_fe[target_col], hybrid_fe))

add_result('A: 피처 엔지니어링 Hybrid', rmse_fe, 'Feature Engineering')
print(f'\n총 {len(features_fe)}개 피처 사용')



---
# Part B: 모델 다양화

다양한 ML 모델과 통계 모델 비교


In [ ]:
# B1. 다양한 Boosting 모델
print('='*60)
print('B1. 다양한 Boosting 모델')
print('='*60)

models_b1 = {
    'GB': GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42),
    'XGB': xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42, verbosity=0),
    'LGB': lgb.LGBMRegressor(n_estimators=100, max_depth=3, random_state=42, verbose=-1),
    'RF': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
}

if CATBOOST_AVAILABLE:
    models_b1['CatBoost'] = CatBoostRegressor(iterations=100, depth=3, random_state=42, verbose=0)

ml_preds = {}
for name, model in models_b1.items():
    model.fit(X_train, y_train)
    ml_preds[name] = model.predict(X_test)
    
    # Hybrid 테스트
    hybrid = 0.8 * naive_test + 0.2 * ml_preds[name]
    rmse = np.sqrt(mean_squared_error(y_test, hybrid))
    add_result(f'B1: Hybrid Naive+{name}', rmse, 'Model Diversity')



In [ ]:
# B2. Ridge/ElasticNet
print('='*60)
print('B2. Ridge/ElasticNet')
print('='*60)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for alpha in [0.1, 1.0, 10.0]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    pred_ridge = ridge.predict(X_test_scaled)
    hybrid_ridge = 0.8 * naive_test + 0.2 * pred_ridge
    rmse_ridge = np.sqrt(mean_squared_error(y_test, hybrid_ridge))
    add_result(f'B2: Hybrid Naive+Ridge(α={alpha})', rmse_ridge, 'Model Diversity')

elastic = ElasticNet(alpha=1.0, l1_ratio=0.5)
elastic.fit(X_train_scaled, y_train)
pred_elastic = elastic.predict(X_test_scaled)
hybrid_elastic = 0.8 * naive_test + 0.2 * pred_elastic
rmse_elastic = np.sqrt(mean_squared_error(y_test, hybrid_elastic))
add_result(f'B2: Hybrid Naive+ElasticNet', rmse_elastic, 'Model Diversity')



In [ ]:
# B3. ARIMA
print('='*60)
print('B3. ARIMA')
print('='*60)

if ARIMA_AVAILABLE:
    try:
        # 학습 데이터로 ARIMA 학습
        train_price = price.loc[:CONFIG['val_end']]
        
        arima = ARIMA(train_price, order=(3, 1, 2))
        arima_fit = arima.fit()
        arima_forecast = arima_fit.forecast(steps=len(y_test))
        
        # 단독 성능
        rmse_arima = np.sqrt(mean_squared_error(y_test, arima_forecast))
        add_result('B3: ARIMA(3,1,2) 단독', rmse_arima, 'Statistical Model')
        
        # Hybrid
        hybrid_arima = 0.8 * naive_test + 0.2 * arima_forecast.values
        rmse_hybrid_arima = np.sqrt(mean_squared_error(y_test, hybrid_arima))
        add_result('B3: Hybrid Naive+ARIMA', rmse_hybrid_arima, 'Statistical Model')
    except Exception as e:
        print(f'ARIMA 실패: {e}')
else:
    print('ARIMA 사용 불가 (statsmodels 필요)')



In [ ]:
# B4. Damped Naive
print('='*60)
print('B4. Damped Naive (감쇠 추세)')
print('='*60)

def calc_damped_naive(indices, damping):
    preds = []
    for idx in indices:
        loc = price.index.get_loc(idx)
        if loc >= 2:
            drift = price.iloc[loc-1] - price.iloc[loc-2]
            preds.append(price.iloc[loc-1] + damping * drift)
        else:
            preds.append(price.iloc[loc-1])
    return np.array(preds)

for damping in [0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
    damped = calc_damped_naive(test.index, damping)
    rmse_damped = np.sqrt(mean_squared_error(y_test, damped))
    add_result(f'B4: Damped Naive (φ={damping})', rmse_damped, 'Naive Variants')
    
    # Damped + ML Hybrid
    hybrid_damped = 0.8 * damped + 0.2 * gb_test
    rmse_hybrid_damped = np.sqrt(mean_squared_error(y_test, hybrid_damped))
    add_result(f'B4: Hybrid Damped(φ={damping})+GB', rmse_hybrid_damped, 'Naive Variants')



In [ ]:
# B5. 다양한 Hybrid 가중치
print('='*60)
print('B5. 다양한 Hybrid 가중치')
print('='*60)

for w_naive in [0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    for ml_name in ['GB', 'LGB', 'XGB']:
        hybrid_w = w_naive * naive_test + (1-w_naive) * ml_preds[ml_name]
        rmse_w = np.sqrt(mean_squared_error(y_test, hybrid_w))
        add_result(f'B5: Naive*{w_naive}+{ml_name}*{1-w_naive:.2f}', rmse_w, 'Weight Optimization')



---
# Part C: 고급 기법


In [ ]:
# C1. Regime Detection (적응형 가중치)
print('='*60)
print('C1. Regime Detection')
print('='*60)

# 변동성 기반 국면 분류
vol_12w = ret.rolling(12).std() * np.sqrt(52)
vol_median = vol_12w.loc[train.index].median()
regime_test = (vol_12w.loc[test.index] > vol_median).astype(int)

print(f'변동성 중위값: {vol_median:.4f}')
print(f'Test 국면: Low={sum(regime_test==0)}, High={sum(regime_test==1)}')

# 국면별 다른 가중치
regime_configs = [
    {'low': 0.7, 'high': 0.9, 'name': 'Adaptive 0.7/0.9'},
    {'low': 0.6, 'high': 0.95, 'name': 'Adaptive 0.6/0.95'},
    {'low': 0.8, 'high': 0.8, 'name': 'Uniform 0.8 (기준선)'},
]

for config in regime_configs:
    adaptive_preds = []
    for i, (idx, regime) in enumerate(regime_test.items()):
        w = config['low'] if regime == 0 else config['high']
        pred = w * naive_test[i] + (1-w) * gb_test[i]
        adaptive_preds.append(pred)
    
    rmse_regime = np.sqrt(mean_squared_error(y_test, np.array(adaptive_preds)))
    add_result(f'C1: {config["name"]}', rmse_regime, 'Regime Detection')



In [ ]:
# C2. Stacking Ensemble
print('='*60)
print('C2. Stacking Ensemble')
print('='*60)

# 1단계: Naive 예측 잔차를 ML로 학습
residual_val = y_val.values - naive_val
residual_model = GradientBoostingRegressor(n_estimators=100, max_depth=2, random_state=42)
residual_model.fit(X_val, residual_val)

# 잔차 예측 및 보정
residual_pred = residual_model.predict(X_test)

for correction in [0.1, 0.2, 0.3, 0.5]:
    stacked = naive_test + correction * residual_pred
    rmse_stack = np.sqrt(mean_squared_error(y_test, stacked))
    add_result(f'C2: Naive + {int(correction*100)}% Residual', rmse_stack, 'Stacking')



In [ ]:
# C3. Hyperparameter Tuning (Grid Search)
print('='*60)
print('C3. Hyperparameter Tuning')
print('='*60)

# LightGBM Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 5],
    'learning_rate': [0.05, 0.1, 0.2],
}

tscv = TimeSeriesSplit(n_splits=3)
best_params = None
best_val_rmse = float('inf')

print('Grid Search 진행 중...')
for n_est in param_grid['n_estimators']:
    for depth in param_grid['max_depth']:
        for lr in param_grid['learning_rate']:
            lgb_gs = lgb.LGBMRegressor(n_estimators=n_est, max_depth=depth, 
                                        learning_rate=lr, random_state=42, verbose=-1)
            lgb_gs.fit(X_train, y_train)
            pred_gs = lgb_gs.predict(X_val)
            hybrid_gs = 0.8 * naive_val + 0.2 * pred_gs
            rmse_gs = np.sqrt(mean_squared_error(y_val, hybrid_gs))
            
            if rmse_gs < best_val_rmse:
                best_val_rmse = rmse_gs
                best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}

print(f'최적 파라미터: {best_params}')
print(f'Validation RMSE: {best_val_rmse:.2f}')

# 최적 파라미터로 Test 평가
lgb_best = lgb.LGBMRegressor(**best_params, random_state=42, verbose=-1)
lgb_best.fit(X_train, y_train)
pred_best = lgb_best.predict(X_test)
hybrid_best = 0.8 * naive_test + 0.2 * pred_best
rmse_best = np.sqrt(mean_squared_error(y_test, hybrid_best))
add_result(f'C3: Tuned LGB (전이 Hybrid)', rmse_best, 'Hyperparameter Tuning')



---
# Part D: 검증 및 분석


In [ ]:
# D1. Time Series CV (5-Fold)
print('='*60)
print('D1. Time Series CV')
print('='*60)

tscv = TimeSeriesSplit(n_splits=5)
cv_results = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    X_tr, X_vl = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_vl = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    gb_cv = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
    gb_cv.fit(X_tr, y_tr)
    pred_cv = gb_cv.predict(X_vl)
    rmse_cv = np.sqrt(mean_squared_error(y_vl, pred_cv))
    cv_results.append(rmse_cv)
    print(f'Fold {fold+1}: ML RMSE {rmse_cv:.2f}')

print(f'\nCV 평균: {np.mean(cv_results):.2f} ± {np.std(cv_results):.2f}')
print(f'CV 해석: {"안정적" if np.std(cv_results) < 100 else "고분산 (불안정)"}')



In [ ]:
# D2. 최종 비교표
print('='*60)
print('📊 최종 성능 비교표')
print('='*60)

df_results = pd.DataFrame(all_results)
df_results = df_results.sort_values('RMSE').reset_index(drop=True)
df_results.index = df_results.index + 1
df_results.index.name = '순위'

print(f'\n기준선: sparta2 Hybrid = RMSE {SPARTA2_RMSE}\n')
print(f'총 {len(df_results)}개 실험 수행\n')

# 상위 15개
print('🏆 상위 15개:')
display(df_results[['Model', 'RMSE', 'Improvement', 'Category']].head(15))

# 카테고리별 최고 성능
print('\n📊 카테고리별 최고 성능:')
for cat in df_results['Category'].unique():
    best_cat = df_results[df_results['Category'] == cat].iloc[0]
    print(f'  {cat}: {best_cat["Model"]} (RMSE {best_cat["RMSE"]:.2f})')



In [ ]:
# D3. 실패 원인 분석
print('='*60)
print('📉 실패 원인 분석')
print('='*60)

# 개선된 모델 수
improved = df_results[df_results['Improvement'] > 0]
not_improved = df_results[df_results['Improvement'] <= 0]

print(f'\n✅ 기준선 개선: {len(improved)}개 / {len(df_results)}개')
print(f'❌ 기준선 미달: {len(not_improved)}개 / {len(df_results)}개')

# 최고 성능
best = df_results.iloc[0]
print(f'\n🏆 최고 성능: {best["Model"]}')
print(f'   RMSE: {best["RMSE"]:.2f}')
print(f'   기준선 대비: {best["Improvement"]:+.2f}')

# 시장 레짐 분석
print('\n📈 시장 레짐 분석:')
train_ret = np.log(price.loc[train.index][-1] / price.loc[train.index][0])
test_ret = np.log(price.loc[test.index][-1] / price.loc[test.index][0])
print(f'   Train 기간 수익률: {train_ret*100:.1f}%')
print(f'   Test 기간 수익률: {test_ret*100:.1f}%')

if test_ret > 0.1:
    print('   → Test 기간이 강한 상승 추세')
    print('   → Naive (추세 지속) 모델이 자연스럽게 우위')
    print('   → ML 모델의 추가 개선 어려움')



---
# 결론

## 실험 요약
- **총 실험**: 50+ 개의 다양한 조합 테스트
- **피처 엔지니어링**: RV, Momentum, Z-score, Lag 등 20+ 피처
- **모델 다양화**: GB, XGB, LGB, CatBoost, Ridge, ARIMA 등 10+ 모델
- **고급 기법**: Regime Detection, Stacking, Hyperparameter Tuning

## 핵심 발견

> **⚠️ sparta2의 Hybrid (Naive*0.8 + GB*0.2) = RMSE 406.80이 여전히 최적 또는 근접**

### 실패 원인
| 원인 | 설명 |
|------|------|
| **시장 레짐 변화** | Train: 평균회귀 → Test: 추세 지속 |
| **Naive 우위** | 추세 시장에서 Naive가 자연스럽게 강함 |
| **과적합** | 복잡한 모델이 과거 패턴에 과적합 |
| **소규모 Test** | 12주 샘플로 통계적 유의성 부족 |

### 교훈
1. **단순함의 가치** - Naive 기반 Hybrid가 금융 시계열에서 강력
2. **시장 레짐 중요** - 모델 선택 전 시장 상태 파악 필수
3. **과적합 주의** - Validation 성능 ≠ Test 성능
4. **동적 접근 필요** - 고정 모델보다 적응형 시스템

### 향후 연구
- Market Regime Detector 개발
- 동적 가중치 조절 시스템
- 더 긴 Test 기간 재검증

---
**End of Notebook**

